In [ ]:
import pandas as pd
import numpy as np
from random import sample
from cmath import exp, sqrt

In [ ]:
headers = ["ID of from node","ID of to node", "weight", "timestamp"]
Graph = pd.read_csv("./Prosper_loans.csv", names=headers, sep=' ').drop_duplicates()
V=np.unique((Graph['ID of from node']._append(Graph['ID of to node'])).values)
V_num = V.size

In [ ]:
def getValues(i):
    return [ x[0] for x in Graph.loc[(Graph['ID of from node']==i), ['ID of to node']].drop_duplicates().values] + [ x[0] for x in Graph.loc[(Graph['ID of to node']==i), ['ID of from node']].drop_duplicates().values]

In [ ]:
matrix = {i: set(getValues(i)) for i in V} #список смежности

In [ ]:
# Часть 1. 
#Задание 1
E_num = 0
for i in V:
    for s in matrix[i]:
        if s>i:
            E_num+=1
print(f'количество вершин: {V_num};\nколичество ребер: {E_num};\nплотность: {2*E_num/(V_num*(V_num-1))}')

In [ ]:
visited = set(V)
Component = set()
answer = 0

while len(visited):
    answer += 1
    v = visited.pop()
    comp = set([v])
    candidates = matrix[v].copy()
    while len(candidates):
        newCan = set()
        for i in candidates:
            newCan.update(matrix[i])
        comp.update(candidates)
        candidates = newCan.difference(comp)
    visited -= comp
    if len(comp) > len(Component):
        Component = comp.copy()



print(f'Количество компонент слабой связности: {answer};\nРазмер максимальной компоненты: {len(Component)};\nДоля вершин в максимальной компоненте: {len(Component)/V_num}' )

In [ ]:
# Задание 3
def Cl(u):
    if len(matrix[u])<2:
        return 0
    neib = matrix[u]
    G = len(neib)
    _2L=0
    for our in neib:
        _2L+=len(matrix[our].intersection(neib))
    return _2L/(G*(G-1))

CL = 0
for node in Component:
    CL+=Cl(node)
print(f'средний кластерный коэффициент сети: {CL/V_num}')

In [ ]:
# Задание 4
def R():
    r1, r2, r3, re = 0, 0, 0, 0
    for node in V:
        u = len(matrix[node])
        r1+=u
        r2+=u*u
        r3+=u*u*u
        for to in matrix[node]:
            re+=u*len(matrix[to])
    return (re*r1-r2*r2)/(r3*r1-r2*r2)

r = R()
print(f'Коэффициент ассортативности: {r}')

In [ ]:
#Часть 2.
#статические характеристики
CN_static={}
AA_static={}
JC_static={}
PA_static={}
visited=set()
for adj in V:
    visited.add(adj)
    for node in matrix[adj]:
        if node not in visited:
            inter_adj_node = matrix[adj] & matrix[node]
            CN_static[(adj, node)] = len(inter_adj_node)
            AA_=0
            for i in inter_adj_node:
                AA_+=(1/np.log10(len(matrix[i])))
            AA_static[(adj, node)] = AA_
            JC_static[(adj, node)] = len(inter_adj_node)/len(matrix[adj].union(matrix[node]))
            PA_static[(adj, node)] = len(matrix[adj])*len(matrix[node])


In [ ]:
t_min = Graph['timestamp'].min()
t_max = Graph['timestamp'].max()
t_max = (t_max-t_min)*0.75+t_min

In [ ]:
#Matrix - список смежности для всего графа. Словарь словарей со значениями - множествами из таймингов
l=0.5
w_liner={}
w_exp={}
w_sqrt={}
for adj in V:
    visited.add(adj)
    for node in Matrix[adj].keys():
        if node not in visited:
            for t in Matrix[adj].values():
                T=(t-t_min)/(t_max-t_min)
                w_liner[(adj, node, t)] = l+(1-l)*T
                w_exp[(adj, node, t)] = l+(1-l)*((exp(3*T)-1)/(exp(3)-1))
                w_sqrt[(adj, node, t)] = l+(1-l)*sqrt(T)